In [1]:
# Import pandas to create DataFrame 
import pandas as pd

#Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [42]:
# Make DataFrame of the given data 
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')

In [36]:
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.031138,0.079892,0.157382,-0.014636,-0.051778,-0.021332,-0.073593,-0.005386,-0.212557,0.099683,...,-0.085248,-0.096007,-0.000766,0.021399,-0.041432,0.094806,45,NaN,1,6
1,0.113040,0.175731,0.217741,-0.196254,-0.010129,-0.030586,0.067114,-0.072412,-0.239192,0.104741,...,-0.090283,-0.053885,-0.010967,0.062209,-0.122958,0.192949,45,NaN,1,6
2,0.048570,0.091281,0.160776,-0.150937,0.020115,0.044117,-0.050092,-0.045661,-0.155332,0.117206,...,-0.021524,-0.008411,-0.006248,0.031468,-0.056915,0.154731,45,NaN,1,6
3,0.039212,0.118388,0.173831,-0.096659,-0.008702,0.061298,0.008974,-0.003277,-0.065046,0.095480,...,-0.071936,-0.023120,-0.007812,0.057600,-0.121892,0.072796,45,NaN,1,6
4,0.056019,0.170639,0.157917,-0.228605,-0.065965,-0.088732,-0.082243,-0.080568,-0.341500,0.142430,...,-0.155621,-0.079447,0.015316,0.127726,-0.151966,0.169634,45,NaN,1,6


In [5]:
#Checking provided data
null_counts = train.isnull().sum()

print(null_counts)

feature_1        0
feature_2        0
feature_3        0
feature_4        0
feature_5        0
              ... 
feature_768      0
label_1          0
label_2        480
label_3          0
label_4          0
Length: 772, dtype: int64


In [55]:
# splitting dependent & Independent features
y = train[["label_1","label_2","label_3","label_4"]]
y_valid = valid[["label_1","label_2","label_3","label_4"]] 
x = train.drop(y, axis=1)
x_valid = valid.drop(y_valid, axis=1)

# Model Checking & Hyper parameter tuning

For KNN, I used Grid Search to do hyper parameter tuning

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import accuracy_score

def try_knn(train_x, train_y, vaid_x, valid_y):
    grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
    gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
    g_res = gs.fit(train_x, train_y)
    print(f"Best Hyperparameters:",gs.best_params_)
    # Get the best k-NN model with the optimal hyperparameters
    best_knn = gs.best_estimator_
    # Evaluate the best model on the test data
    accuracy = best_knn.score(vaid_x, valid_y)
    print(f"Accuracy for KNN {accuracy}")      
        


Due to performance restriction of my machine, I do hyperparameter tuning manually for SVM.

In [8]:
from sklearn import svm

def try_svm(train_x, train_y, vaid_x, valid_y):
    clf = svm.SVC()
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with default settings (RBF - exponential kernal) ", accuracy_score(valid_y, y_pred))
    
#     clf = svm.LinearSVC(dual="auto")
#     clf.fit(train_x, train_y)
#     X_valid_contiguous = np.ascontiguousarray(vaid_x)
#     y_pred = clf.predict(X_valid_contiguous)
#     print(f"Accuracy Score of Linear SVM (one-vs-the-rest) ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='linear')
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with linear kernal function ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='sigmoid')
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with sigmoid kernal function ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='poly',degree =2)
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with polynomial kernal function with degree 2 ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='poly',degree =2)
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with polynomial kernal function with degree 3 ", accuracy_score(valid_y, y_pred))  

Logistic Regression Model
Checking Hyper Parameter tuning for Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

def try_logistic(train_x, train_y, vaid_x, valid_y):
    for i in ["lbfgs","newton-cg","sag","saga"]:
        logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        logistic_regression.fit(train_x,train_y)
        y_pred = logistic_regression.predict(vaid_x)
        print(f"Accuracy Score of Logistic Regression with solver {i}", accuracy_score(valid_y, y_pred))

Naive Bayes 
Trying out different distributions

In [63]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

def try_naive(train_x, train_y, vaid_x, valid_y):
    
    min_value = np.min(train_x)
    scaled_train_x = train_x - min_value
    scaled_vaid_x = vaid_x - min_value
    
    clf = GaussianNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with Gaussian Naive Bayes", accuracy_score(valid_y, y_pred))
    
    clf = MultinomialNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Naive Bayes with MultinomialNB", accuracy_score(valid_y, y_pred))
    
    clf = ComplementNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with ComplementNB", accuracy_score(valid_y, y_pred))
    
#     clf = CategoricalNB()
#     clf.fit(scaled_train_x, train_y)
#     y_pred = clf.predict(scaled_vaid_x)
#     print(f"Accuracy Score of Naive Bayes with CategoricalNB", accuracy_score(valid_y, y_pred))
    
    clf = BernoulliNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with BernoulliNB", accuracy_score(valid_y, y_pred))  
    



DecisionTree Classifier

In [11]:
from sklearn.tree import DecisionTreeClassifier

def try_decisionTree(train_x, train_y, vaid_x, valid_y):
    # Initialize and train the Decision Tree classifier
    clf = DecisionTreeClassifier()
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Decision Tree", accuracy_score(valid_y, y_pred))

Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

def try_random_forest(train_x, train_y, vaid_x, valid_y):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Random Forest", accuracy_score(valid_y, y_pred))

XG Boost

In [13]:
#installing XG Boost
!pip install xgboost

DEPRECATION: h2o-mlops-client main has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of h2o-mlops-client or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


def try_xgBoost(train_x, train_y, vaid_x, valid_y):
    # Create an XGBoost classifier for multi-class classification
    clf = xgb.XGBClassifier(
    objective='multi:softmax',  # Set the objective for multi-class classification
    num_class=len(np.unique(y)),  # Number of classes
    random_state=42
    )
    le = LabelEncoder()
    train_y = le.fit_transform(train_y)
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    y_pred = le.inverse_transform(y_pred)
    print(f"Accuracy Score of XG Boost", accuracy_score(valid_y, y_pred))

# Feature Engineering Using Mutual Info Classification

In [15]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

def select_cols_using_mutual_info_regression(x,y,n) :
    selected_columns = SelectKBest(mutual_info_classif, k=n)
    selected_columns.fit(x, y)
    return x.columns[selected_columns.get_support()]

In [25]:
selected_cols = select_cols_using_mutual_info_regression(x,y['label_1'],300)

In [26]:
x = x[selected_cols]
x_valid = x_valid[selected_cols]

# Label 1

In [20]:
#Doing Hyper parameter tuning for KNN & checking the best accuracy for label 1
try_knn(x,y["label_1"],x_valid,y_valid["label_1"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 5, 'weights': 'distance'}
Accuracy for KNN 0.24266666666666667


In [ ]:
#Checking SVM Values for Label 1
try_svm(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.016
Accuracy Score of SVM with linear kernal function  0.176


In [ ]:
#Checking Logistic Regression Values for Label1. It provided very good results. 
try_logistic(x,y["label_1"],x_valid,y_valid["label_1"])

In [21]:
#It gave bad results
try_naive(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.016
Accuracy Score of Naive Bayes with MultinomialNB 0.016
Accuracy Score of Naive Bayes with ComplementNB 0.05333333333333334


IndexError: index 1 is out of bounds for axis 1 with size 1

In [27]:
try_decisionTree(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Decision Tree 0.03333333333333333


In [28]:
try_random_forest(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Random Forest 0.056


In [29]:
try_xgBoost(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of XG Boost 0.064


When considering validation accuracy results of label 1, Linear SVM and Logistic Regression gave good results, We can use Linear SVM model for our test data set.

In [33]:
# Test dataset
test = pd.read_csv('test.csv')
test = test.drop(["ID"], axis=1)
test.head()
# test["ID"]

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768
0,0.124623,0.196628,0.257004,-0.156045,-0.054916,0.006071,-0.035149,-0.092019,-0.196302,0.077971,...,-0.221466,0.140292,0.123622,-0.175572,-0.107030,-0.087621,-0.026501,0.139337,-0.083030,0.059507
1,0.109655,0.170158,0.227644,-0.127088,-0.044476,-0.046852,-0.090026,-0.061321,-0.227288,0.066863,...,-0.204930,0.110203,0.085665,-0.286787,-0.113195,-0.057312,-0.055680,0.143939,-0.045760,0.106113
2,0.014854,0.030051,0.115092,-0.017179,0.002720,-0.011692,-0.078855,-0.042991,-0.096283,0.042701,...,-0.032937,0.075821,0.030987,-0.149850,-0.003155,-0.010207,-0.001427,0.000934,-0.017069,0.048123
3,0.196893,0.113314,0.352175,-0.108499,-0.064472,-0.073239,-0.086402,0.008671,-0.342217,0.104941,...,-0.255167,0.096579,0.069413,-0.215386,-0.075168,-0.035071,-0.023375,0.067768,-0.181530,0.174444
4,0.033004,0.013373,0.124001,-0.016143,0.010120,0.010635,-0.055789,-0.036282,-0.059422,0.060278,...,-0.035814,0.093764,0.027321,-0.116009,0.010096,-0.042293,0.005347,0.007722,-0.007731,0.058799


In [40]:
x.shape

(28520, 300)

In [43]:
# taken model for this lable is 
# Fitting 3 folds for each of 8 candidates, totalling 24 fits
# Best Hyperparameters: {'n_neighbors': 5, 'weights': 'distance'}
# Accuracy for KNN 0.24266666666666667

grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = gs.fit(x[300:], y["label_1"][300:])
print(f"Best Hyperparameters:",gs.best_params_)
# Get the best k-NN model with the optimal hyperparameters
best_knn = gs.best_estimator_
# Evaluate the best model on the test data
accuracy = best_knn.score(x_valid,y_valid["label_1"])
print(f"Accuracy for KNN {accuracy}")     

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_1': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_1.csv', index=False)
print(pridiction)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 5, 'weights': 'distance'}
Accuracy for KNN 0.24
[26 18 16  7 58 46  7 22 29 26 33 32 54 51 22 48 23  2 43 42 11 35 39 34
 21 13 28 55  2 51 55 16  1 56 30 29 31  6 51 18 19 47  6 46 59 25 31 18
  5 32 25 56 39 37 32 40  5 52 28 34 32 47 55 24 23 18 20  6 36 38 22 26
 44 46 55 36 49 53 57 12 52 38 23 27 33 49 14 19 12 39 36 43 34 32 54 46
 27  5  7 42 19 26 31 36 53 10 12 38 38  2 57 14  7 50 46 46 17 34 30 40
 48 40  8 22 38 15 13  9 52  5 40 13 10 55 28 25 42 57 16 27 51 21 51 27
 32 33 16  3 11 46 42 37 47 50  3 48 19 20 15  8  5 18  5 28 46 60 50 40
 17 32 45 57 42 16 11 35 34 52 41 21  7  8 21 19 37 55 20 28 23 35 46 52
 30 27  1  8 28 37  3 36 17 60 18  3 22  9 43 25 17 26 19 36 36 21  1 58
 39 47 38  6 50 39 23  5 48 22 58 15 30 38 18 54  8 41 16  8 43 21  8 52
 10 16 47 20 15 51 56 11 16 48 50 51 10 51 31  6  9 31 57 22  5 11 11 43
  8  7 58 23 50 58 13 13 47 50 60 50 47 36 28 36 53

# Label 3

In [44]:
try_knn(x,y["label_3"],x_valid,y_valid["label_3"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'uniform'}
Accuracy for KNN 0.8266666666666667


In [45]:
try_logistic(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Logistic Regression with solver lbfgs 0.8773333333333333
Accuracy Score of Logistic Regression with solver newton-cg 0.8773333333333333
Accuracy Score of Logistic Regression with solver sag 0.8773333333333333
Accuracy Score of Logistic Regression with solver saga 0.8773333333333333


In [46]:
try_svm(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.8106666666666666
Accuracy Score of SVM with linear kernal function  0.8653333333333333
Accuracy Score of SVM with sigmoid kernal function  0.7306666666666667
Accuracy Score of SVM with polynomial kernal function with degree 2  0.8706666666666667
Accuracy Score of SVM with polynomial kernal function with degree 3  0.8706666666666667


In [47]:
try_naive(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.8106666666666666
Accuracy Score of Naive Bayes with MultinomialNB 0.8106666666666666
Accuracy Score of Naive Bayes with ComplementNB 0.816


IndexError: index 1 is out of bounds for axis 1 with size 1

In [48]:
try_decisionTree(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Decision Tree 0.5053333333333333


In [49]:
try_random_forest(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Random Forest 0.47333333333333333


In [50]:
try_xgBoost(x,y["label_3"],x_valid,y_valid["label_3"])

ValueError: y contains previously unseen labels: [2]

When considering validation accuracy results of label 3, Logistic Regression, SVM provided good accuracy. Since we got good accuracy close to 100%, We can use Logistics Regression for our test data set.

In [51]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.8773333333333333
# try_logistic(x,y["label_3"],x_valid,y_valid["label_3"])
clf = svm.SVC(kernel='linear')

clf.fit(x, y["label_3"])
X_valid_contiguous = np.ascontiguousarray(x_valid)
y_pred = clf.predict(X_valid_contiguous)
print(f"Accuracy Score of SVM with linear kernal function ", accuracy_score(y_valid["label_3"], y_pred))

pridiction = clf.predict(test)
df = pd.DataFrame({'label_3': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_3.csv', index=False)
print(pridiction)


Accuracy Score of SVM with linear kernal function  0.8653333333333333
[0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0
 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1

# Label 4

# Handling Class Imbalance using SMOTE 

In [64]:
y["label_4"].value_counts()

6     19938
2      1449
0       955
12      954
7       938
13      482
1       481
11      480
10      480
3       479
5       478
9       472
4       469
8       465
Name: label_4, dtype: int64

In [65]:
pip install imbalanced-learn


DEPRECATION: h2o-mlops-client main has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of h2o-mlops-client or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [56]:
try_knn(x,y["label_4"],x_valid,y_valid["label_4"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
Accuracy for KNN 0.676


In [57]:
try_logistic(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Logistic Regression with solver lbfgs 0.10933333333333334
Accuracy Score of Logistic Regression with solver newton-cg 0.10933333333333334
Accuracy Score of Logistic Regression with solver sag 0.10933333333333334
Accuracy Score of Logistic Regression with solver saga 0.10933333333333334


In [58]:
try_svm(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.7093333333333334
Accuracy Score of SVM with linear kernal function  0.06266666666666666
Accuracy Score of SVM with sigmoid kernal function  0.396
Accuracy Score of SVM with polynomial kernal function with degree 2  0.10666666666666667
Accuracy Score of SVM with polynomial kernal function with degree 3  0.10666666666666667


In [59]:
try_naive(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.02266666666666667
Accuracy Score of Naive Bayes with MultinomialNB 0.5533333333333333
Accuracy Score of Naive Bayes with ComplementNB 0.24666666666666667


IndexError: index 1 is out of bounds for axis 1 with size 1

In [60]:
try_decisionTree(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Decision Tree 0.12533333333333332


In [61]:
try_random_forest(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Random Forest 0.6866666666666666


In [62]:
try_xgBoost(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of XG Boost 0.33066666666666666


When considering validation accuracy results of label 4, We can use Random Forest method

In [66]:
# taken model for this lable is 
# Accuracy Score of Random Forest 0.6866666666666666
# try_random_forest(x,y["label_4"],x_valid,y_valid["label_4"])

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(x, y["label_4"])
y_pred = clf.predict(x_valid)
print(f"Accuracy Score of Random Forest", accuracy_score(y_valid["label_4"], y_pred))    

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_4': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_4.csv', index=False)
print(pridiction)


Accuracy Score of Random Forest 0.6866666666666666
[26 18 16  7 58 46  7 22 29 26 33 32 54 51 22 48 23  2 43 42 11 35 39 34
 21 13 28 55  2 51 55 16  1 56 30 29 31  6 51 18 19 47  6 46 59 25 31 18
  5 32 25 56 39 37 32 40  5 52 28 34 32 47 55 24 23 18 20  6 36 38 22 26
 44 46 55 36 49 53 57 12 52 38 23 27 33 49 14 19 12 39 36 43 34 32 54 46
 27  5  7 42 19 26 31 36 53 10 12 38 38  2 57 14  7 50 46 46 17 34 30 40
 48 40  8 22 38 15 13  9 52  5 40 13 10 55 28 25 42 57 16 27 51 21 51 27
 32 33 16  3 11 46 42 37 47 50  3 48 19 20 15  8  5 18  5 28 46 60 50 40
 17 32 45 57 42 16 11 35 34 52 41 21  7  8 21 19 37 55 20 28 23 35 46 52
 30 27  1  8 28 37  3 36 17 60 18  3 22  9 43 25 17 26 19 36 36 21  1 58
 39 47 38  6 50 39 23  5 48 22 58 15 30 38 18 54  8 41 16  8 43 21  8 52
 10 16 47 20 15 51 56 11 16 48 50 51 10 51 31  6  9 31 57 22  5 11 11 43
  8  7 58 23 50 58 13 13 47 50 60 50 47 36 28 36 53 39 34 32  5  2 19 53
 57 50 30 32 41 37 41 51 20 20 31 21 44 31 15 40 46 39 21 25 31 44 54 34


# Label 2

# Handling Missing Values 

Since i am handling each feature separately, I thought removing the rows which have empty values for label 2, is the best way to handle the missing values for label 2 as no artifical entries will not be added.

In [67]:
data_label_2 = train.dropna(subset=['label_2'])
valid_label_2 = valid.dropna(subset=['label_2'])
y_label_2 = data_label_2[["label_1","label_2","label_3","label_4"]] 
x_label_2 = data_label_2.drop(y_label_2, axis=1)
y_label_2_valid = valid_label_2[["label_1","label_2","label_3","label_4"]] 
x_label_2_valid = valid_label_2.drop(y_label_2_valid, axis=1)

In [68]:
try_knn(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
Accuracy for KNN 0.27445652173913043


In [69]:
try_logistic(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Logistic Regression with solver lbfgs 0.17119565217391305
Accuracy Score of Logistic Regression with solver newton-cg 0.17119565217391305
Accuracy Score of Logistic Regression with solver sag 0.17119565217391305
Accuracy Score of Logistic Regression with solver saga 0.17119565217391305


In [70]:
try_svm(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.12092391304347826
Accuracy Score of SVM with linear kernal function  0.13179347826086957
Accuracy Score of SVM with sigmoid kernal function  0.06521739130434782
Accuracy Score of SVM with polynomial kernal function with degree 2  0.1480978260869565
Accuracy Score of SVM with polynomial kernal function with degree 3  0.1480978260869565


In [71]:
try_naive(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.06114130434782609
Accuracy Score of Naive Bayes with MultinomialNB 0.08695652173913043
Accuracy Score of Naive Bayes with ComplementNB 0.11277173913043478
Accuracy Score of Naive Bayes with BernoulliNB 0.025815217391304348


In [72]:
try_decisionTree(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Decision Tree 0.09510869565217392


In [73]:
try_random_forest(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Random Forest 0.12092391304347826


In [74]:
try_xgBoost(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of XG Boost 0.11413043478260869


Here KNN model performed well beating other models. So here we can use KNN model to predict label 2 data points.

In [75]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.9746666666666667
# try_knn(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])
# Fitting 3 folds for each of 8 candidates, totalling 24 fits
# Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
# Accuracy for KNN 0.27445652173913043



grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = gs.fit(x_label_2, y_label_2["label_2"])
print(f"Best Hyperparameters:",gs.best_params_)
# Get the best k-NN model with the optimal hyperparameters
best_knn = gs.best_estimator_
# Evaluate the best model on the test data
accuracy = best_knn.score(x_label_2_valid,y_label_2_valid["label_2"])
print(f"Accuracy for KNN {accuracy}")     

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_2': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_2.csv', index=False)
print(pridiction)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
Accuracy for KNN 0.27445652173913043
[22. 25. 30. 27. 29. 28. 27. 33. 23. 22. 26. 23. 27. 26. 33. 26. 28. 25.
 31. 29. 33. 24. 29. 25. 26. 27. 28. 23. 25. 26. 23. 30. 30. 24. 28. 23.
 26. 25. 26. 25. 23. 23. 25. 30. 31. 22. 26. 25. 25. 23. 22. 24. 29. 27.
 23. 26. 30. 34. 28. 25. 23. 23. 23. 26. 28. 25. 25. 25. 22. 32. 33. 22.
 61. 30. 26. 26. 26. 24. 27. 26. 34. 32. 28. 31. 26. 26. 31. 23. 26. 29.
 22. 31. 25. 23. 27. 30. 31. 25. 27. 29. 23. 22. 26. 22. 24. 36. 26. 32.
 32. 25. 27. 31. 26. 24. 30. 30. 26. 25. 28. 26. 26. 26. 41. 33. 32. 28.
 27. 35. 34. 28. 26. 27. 36. 23. 28. 22. 29. 27. 30. 31. 30. 26. 23. 31.
 23. 30. 30. 31. 33. 30. 29. 27. 23. 24. 31. 26. 23. 25. 28. 41. 25. 25.
 25. 28. 30. 27. 24. 26. 26. 23. 26. 27. 29. 30. 33. 24. 25. 34. 30. 23.
 27. 41. 26. 23. 27. 23. 61. 28. 28. 24. 30. 34. 28. 31. 30. 41. 28. 27.
 31. 22. 26. 27. 25. 31. 33. 35. 31.

## Combine all CSV files for submission

In [76]:
label_1_o = pd.read_csv('label_1.csv', header=None)[1:]
label_2_o = pd.read_csv('label_2.csv', header=None)[1:]
label_3_o = pd.read_csv('label_3.csv', header=None)[1:]
label_4_o = pd.read_csv('label_4.csv', header=None)[1:]

In [77]:
label_3_o

,0
1,0
2,1
3,1
4,1
5,0
...,...
740,1
741,1
742,1
743,1


In [78]:
combined_df = pd.concat([label_1_o, label_2_o, label_3_o, label_4_o], axis=1)

column_names = ["label_1", "label_2", "label_3", "label_4"]  # Replace with your desired column names
combined_df.columns = column_names

combined_df['ID'] = range(1, len(combined_df) + 1)

column_order = ['ID',"label_1", "label_2", "label_3", "label_4"]

# Use reindex to reorder the columns
combined_df = combined_df.reindex(columns=column_order)

combined_df.to_csv("combined_data.csv",index=False)
combined_df

,ID,label_1,label_2,label_3,label_4
1,1,26,22.0,0,26
2,2,18,25.0,1,18
3,3,16,30.0,1,16
4,4,7,27.0,1,7
5,5,58,29.0,0,58
...,...,...,...,...,...
740,740,35,24.0,1,35
741,741,35,24.0,1,35
742,742,55,30.0,1,55
743,743,38,32.0,1,38


In [79]:
combined_csv = pd.read_csv("combined_data.csv")
combined_csv.head()

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,26
1,2,18,25.0,1,18
2,3,16,30.0,1,16
3,4,7,27.0,1,7
4,5,58,29.0,0,58


In [80]:
combined_csv

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,26
1,2,18,25.0,1,18
2,3,16,30.0,1,16
3,4,7,27.0,1,7
4,5,58,29.0,0,58
...,...,...,...,...,...
739,740,35,24.0,1,35
740,741,35,24.0,1,35
741,742,55,30.0,1,55
742,743,38,32.0,1,38
